In [ ]:
import cv2
import numpy as np
import sklearn
from random import shuffle


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = currentPath='./testDrive/IMG/'+batch_sample[0].split('/')[-1]
                #print(name)
                center_image = cv2.imread(name)
                center_image=(center_image/255)-0.5
                vv = center_image[75:304, 0:319] 
                #plt.imshow(vv)
                #plt.show()
                center_angle = float(batch_sample[3])
                #center_image=scipy.misc.imresize(center_image, (200, 320,3))                
                
                images.append(vv)
                angles.append(center_angle)
                images.append(cv2.flip(vv,1))
                angles.append(center_angle*-0.7)
                #name = './testDrive/IMG/'+batch_sample[1].split('/')[-1]
                #left_image = cv2.imread(name)
                #left_angle = float(batch_sample[3])
                #images.append(left_image)
                #angles.append(left_angle)
                #images.append(cv2.flip(left_image,1))
                #angles.append(left_angle*-0.7)
                #name = './testDrive/IMG/'+batch_sample[2].split('/')[-1]
                #right_image = cv2.imread(name)
                #right_angle = float(batch_sample[3])
                #images.append(right_image)
                #angles.append(right_angle)
                #images.append(cv2.flip(right_image,1))
                #angles.append(right_angle*-0.7)
                
            print(len(images))

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            sklearn.utils.shuffle(X_train, y_train)
            #print(len(X_train))
            yield (X_train, y_train)



In [ ]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


lines=[]
with open ('./testDrive/driving_log.csv') as csvfile:
    next(csvfile)
    reader =csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images=[]
mesurements=[]


train_samples, validation_samples = train_test_split(lines, test_size=0.2) 
print(len(lines))
    
train_generator = generator(train_samples)
validation_generator = generator(validation_samples) 

#ch, row, col = 3,160, 320  

import json
from keras.models import  Model

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D,Lambda,Cropping2D,Input,Dropout
from keras import backend as K

input1=Input(shape=(85,319,3))


#normI=Lambda(lambda x: ((x/255)-0.5) )
#croppedI=Cropping2D(cropping=((75,14),(0,0)),input_shape=(row,col,ch))
#normI=Lambda(lambda x: ((x/255)-0.5), input_shape=(65,col,ch) )(croppedI)
 
m1=InceptionV3(weights="imagenet",include_top=False)
x=m1(input1)
#x=m1.output
#x = Dense(1024, activation='relu')(x)


x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(256, activation='relu')(x)
#x = Dropout(0.5)(x)
predict = Dense(1)(x)

model = Model(inputs=input1, outputs=predict)
for layer in m1.layers:
    layer.trainable = False

for i, layer in enumerate(m1.layers):
    print(i, layer.name)
for layer in model.layers[:289]:
    layer.trainable = False
for layer in model.layers[289:]:
    layer.trainable = True

model.compile(loss='mse',optimizer='adam')

model.fit_generator(train_generator,
            steps_per_epoch=len(train_samples),
            nb_epoch=1,
            validation_data=validation_generator,
            nb_val_samples=len(validation_samples)
            )



model.save('model.h5')